In [169]:
import random
import pandas as pd
import matplotlib.pyplot as plt

data=pd.read_excel('LQ.xlsx')
data.info()
grp=data.groupby('Area').count

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2198 entries, 0 to 2197
Data columns (total 6 columns):
Area          2198 non-null object
RSSI          2198 non-null int64
SQ            2198 non-null int64
ETX           2198 non-null float64
Distance      2198 non-null float64
Throughput    2198 non-null float64
dtypes: float64(3), int64(2), object(1)
memory usage: 103.1+ KB


In [170]:
#Getting the maximum throughput in order to calculate TPR
maximum=data.groupby(['Area'])['Throughput'].max()
maximum

Area
ParkLOS           17.760023
ParkNLOS          17.929483
RESIDENTIAL_IN    18.342404
RESIDENTIAL_OD    14.281408
TrackLOS          16.268372
TrackNLOS         15.889274
Name: Throughput, dtype: float64

In [171]:
data.head()

,Area,RSSI,SQ,ETX,Distance,Throughput
0,ParkLOS,81,100,1.000000,0.000000,13.663293
1,TrackLOS,79,100,1.111111,15.321800,12.887519
2,ParkNLOS,44,78,1.000000,49.807500,5.449338
3,RESIDENTIAL_IN,72,100,1.000000,0.000000,12.904800
4,ParkLOS,78,100,1.000000,0.185362,13.563479


In [172]:
data['TPR']=0
data.loc[data.Area=='ParkLOS', ['TPR']]=data['Throughput']/maximum.ParkLOS
data.loc[data.Area=='ParkNLOS', ['TPR']]=data['Throughput']/maximum.ParkNLOS
data.loc[data.Area=='RESIDENTIAL_OD', ['TPR']]=data['Throughput']/maximum.RESIDENTIAL_OD
data.loc[data.Area=='RESIDENTIAL_IN', ['TPR']]=data['Throughput']/maximum.RESIDENTIAL_IN

data.loc[data.Area=='TrackLOS', ['TPR']]=data['Throughput']/maximum.TrackLOS
data.loc[data.Area=='TrackNLOS', ['TPR']]=data['Throughput']/maximum.TrackNLOS

data.describe()
data.head()

,Area,RSSI,SQ,ETX,Distance,Throughput,TPR
0,ParkLOS,81,100,1.000000,0.000000,13.663293,0.769329
1,TrackLOS,79,100,1.111111,15.321800,12.887519,0.792182
2,ParkNLOS,44,78,1.000000,49.807500,5.449338,0.303932
3,RESIDENTIAL_IN,72,100,1.000000,0.000000,12.904800,0.703550
4,ParkLOS,78,100,1.000000,0.185362,13.563479,0.763708


In [173]:
encoded_data=pd.get_dummies(data)
encoded_data.head()

,RSSI,SQ,ETX,Distance,Throughput,TPR,Area_ParkLOS,Area_ParkNLOS,Area_RESIDENTIAL_IN,Area_RESIDENTIAL_OD,Area_TrackLOS,Area_TrackNLOS
0,81,100,1.000000,0.000000,13.663293,0.769329,1,0,0,0,0,0
1,79,100,1.111111,15.321800,12.887519,0.792182,0,0,0,0,1,0
2,44,78,1.000000,49.807500,5.449338,0.303932,0,1,0,0,0,0
3,72,100,1.000000,0.000000,12.904800,0.703550,0,0,1,0,0,0
4,78,100,1.000000,0.185362,13.563479,0.763708,1,0,0,0,0,0


In [183]:
feature=['RSSI', 'SQ','ETX','Distance','Area_ParkLOS','Area_ParkNLOS','Area_RESIDENTIAL_IN','Area_RESIDENTIAL_OD','Area_TrackLOS','Area_TrackNLOS']
X=encoded_data[feature]
y=encoded_data['TPR']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X, y, random_state=100,test_size=0.20)
X_train.shape[1]
X_test.head()

,RSSI,Area_ParkLOS,SQ,Area_ParkNLOS,Area_RESIDENTIAL_IN,Area_RESIDENTIAL_OD,Distance,Area_TrackLOS,Area_TrackNLOS,ETX
1663,49,0,93,0,0,1,91.5625,0,0,1.000000
1699,44,0,63,0,0,1,102.8125,0,0,1.000000
960,54,0,99,0,0,0,36.0386,1,0,1.000000
1434,65,0,100,0,0,1,20.0000,0,0,1.000000
1634,51,0,94,0,0,1,82.5000,0,0,1.111111


In [184]:
#Normalization
feature1=['RSSI', 'SQ','ETX','Distance']
mean=X_train[feature1].mean(axis=0)
std=X_train[feature1].std(axis=0)

#Normalization of train dataset
X_train[feature1]=X_train[feature1]-mean
X_train[feature1]=X_train[feature1]/std

#Normalization of test dataset
X_test[feature1]=X_test[feature1]-mean
X_test[feature1]=X_test[feature1]/std
std

RSSI        15.415087
SQ          12.226793
ETX          0.161828
Distance    31.487873
dtype: float64

In [69]:
feature1=['RSSI', 'SQ','ETX','Distance']


RSSI        1
SQ          7
ETX         1
Distance    1
dtype: int64

In [185]:
import numpy as np
#reprocessing the data for AE
feature1=['RSSI', 'SQ','ETX','Distance']
maxi=X_train[feature1].max(axis=0) 

#Scaling of train dataset
X_train[feature1]=X_train[feature1]/maxi

mini=X_train[feature1].min(axis=0)

#NScaling of test dataset
X_test[feature1]=X_test[feature1]/maxi
mini_test=X_test[feature1].min(axis=0)

#Scaleup of  dataset
scale_values=[1, 7, 1, 1]
scalar = pd.Series(scale_values, index=feature1)
X_train[feature1]=X_train[feature1]+scalar
X_test[feature1]=X_test[feature1]+scalar


maxi_1=X_train.max(axis=0)
X_train=X_train/maxi_1
X_test=X_test/maxi_1

X_test.head()

,RSSI,Area_ParkLOS,SQ,Area_ParkNLOS,Area_RESIDENTIAL_IN,Area_RESIDENTIAL_OD,Distance,Area_TrackLOS,Area_TrackNLOS,ETX
1663,0.404076,0.0,0.894777,0.0,0.0,1.0,0.815268,0.0,0.0,0.494939
1699,0.345652,0.0,0.443823,0.0,0.0,1.0,0.878605,0.0,0.0,0.494939
960,0.462500,0.0,0.984968,0.0,0.0,0.0,0.502672,1.0,0.0,0.494939
1434,0.591032,0.0,1.000000,0.0,0.0,1.0,0.412376,0.0,0.0,0.494939
1634,0.427445,0.0,0.909809,0.0,0.0,1.0,0.764247,0.0,0.0,0.507257


In [186]:
minit=X_test[feature1].min(axis=0)
minit

RSSI        0.240489
SQ          0.052996
ETX         0.494939
Distance    0.299778
dtype: float64

In [194]:
from keras.layers import Input, Dense
from keras.models import Model
import numpy
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

#size of the encoder
encoding_dim=4

#input shape
input_img=Input(shape=(10,))

#encoded
encoded=Dense(encoding_dim,activation='relu',kernel_initializer='normal')(input_img)

#decoded
decoded=Dense(10,activation='relu',kernel_initializer='normal')(encoded)

# this model maps an input to its reconstruction
autoencoder=Model(input_img, decoded)


# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)


# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))

# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]

# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam', loss='mse', metrics=['mae'])

autoencoder.fit(X_train, X_train,
                epochs=150,
                batch_size=25,
                shuffle=True)

Epoch 1/150
1758/1758 [==============================] - 1s 293us/step - loss: 0.2309 - mean_absolute_error: 0.3208
Epoch 2/150
1758/1758 [==============================] - 0s 32us/step - loss: 0.1446 - mean_absolute_error: 0.2447
Epoch 3/150
1758/1758 [==============================] - 0s 25us/step - loss: 0.0999 - mean_absolute_error: 0.1796
Epoch 4/150
1758/1758 [==============================] - 0s 38us/step - loss: 0.0918 - mean_absolute_error: 0.1615
Epoch 5/150
1758/1758 [==============================] - 0s 32us/step - loss: 0.0867 - mean_absolute_error: 0.1533
Epoch 6/150
1758/1758 [==============================] - 0s 27us/step - loss: 0.0818 - mean_absolute_error: 0.1453
Epoch 7/150
1758/1758 [==============================] - 0s 33us/step - loss: 0.0771 - mean_absolute_error: 0.1367
Epoch 8/150
1758/1758 [==============================] - 0s 27us/step - loss: 0.0730 - mean_absolute_error: 0.1285
Epoch 9/150
1758/1758 [==============================] - 0s 36us/step - loss: 0

In [193]:
encoded_imgs = encoder.predict(X_test)
decoded_imgs = decoder.predict(encoded_imgs)
decoded_imgs[0]

array([  0.      ,   0.      ,  23.084112,   0.      ,   0.      ,
         0.      , 236.45131 ,   0.      ,   0.      ,   0.      ],
      dtype=float32)

In [164]:
scalar

RSSI        1
SQ          7
ETX         1
Distance    1
dtype: int64

In [187]:
X_test.head()

,RSSI,Area_ParkLOS,SQ,Area_ParkNLOS,Area_RESIDENTIAL_IN,Area_RESIDENTIAL_OD,Distance,Area_TrackLOS,Area_TrackNLOS,ETX
1663,0.404076,0.0,0.894777,0.0,0.0,1.0,0.815268,0.0,0.0,0.494939
1699,0.345652,0.0,0.443823,0.0,0.0,1.0,0.878605,0.0,0.0,0.494939
960,0.462500,0.0,0.984968,0.0,0.0,0.0,0.502672,1.0,0.0,0.494939
1434,0.591032,0.0,1.000000,0.0,0.0,1.0,0.412376,0.0,0.0,0.494939
1634,0.427445,0.0,0.909809,0.0,0.0,1.0,0.764247,0.0,0.0,0.507257


## Transform Data to Original Form

In [188]:
#conversion
X_test=X_test*maxi_1

X_test[feature1]=X_test[feature1]-scalar

X_test[feature1]=X_test[feature1]*maxi

X_test[feature1]=X_test[feature1]*std

X_test[feature1]=X_test[feature1]+mean

X_test.head()

,RSSI,Area_ParkLOS,SQ,Area_ParkNLOS,Area_RESIDENTIAL_IN,Area_RESIDENTIAL_OD,Distance,Area_TrackLOS,Area_TrackNLOS,ETX
1663,49.0,0.0,93.0,0.0,0.0,1.0,91.5625,0.0,0.0,1.000000
1699,44.0,0.0,63.0,0.0,0.0,1.0,102.8125,0.0,0.0,1.000000
960,54.0,0.0,99.0,0.0,0.0,0.0,36.0386,1.0,0.0,1.000000
1434,65.0,0.0,100.0,0.0,0.0,1.0,20.0000,0.0,0.0,1.000000
1634,51.0,0.0,94.0,0.0,0.0,1.0,82.5000,0.0,0.0,1.111111
